In [1]:
model = "google/gemma-2b"
device = "cuda"
batch_size = 64
from transformer_lens import HookedTransformer

tl_model = HookedTransformer.from_pretrained_no_processing(model, device=device)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model google/gemma-2b into HookedTransformer


In [1]:
import torch as th
from tqdm.auto import tqdm

_ = th.set_grad_enabled(False)
# Parameters
model = "google/gemma-2b"
device = "cuda"
batch_size = 64
from transformer_lens import HookedTransformer

tl_model = HookedTransformer.from_pretrained_no_processing(model, device=device)
from transformers import AutoModelForCausalLM, AutoTokenizer

hf_model = AutoModelForCausalLM.from_pretrained(model, device_map=device)
hf_tokenizer = AutoTokenizer.from_pretrained(model)

p = '''Français: "mot" - 中文: "詞"
Français: "position" - 中文: "地理位置"
Français: "porte" - 中文: "戶"
Français: "plate-forme" - 中文: "月台"
Français: "onde" - 中文: "海浪"
Français: "temps" - 中文: "'''
prompts = []
for i in range(6):
    prompts.append(p)
    print(p)
    print("")
    p = '"'.join("\n".join(p.split("\n")[:-1]).split('"')[:-2]) + '"'

tl_probs = []

from torch.utils.data import DataLoader

data = DataLoader(
    prompts,
    batch_size=batch_size,
    shuffle=False,
)
for batch in tqdm(data):
    tl_probs.append(tl_model(batch)[:, -1].cpu())
tl_probs = th.cat(tl_probs)
tl_probs = th.softmax(tl_probs, dim=-1)

hf_probs = []
for batch in tqdm(data):
    hf_probs.append(
        hf_model(**hf_tokenizer(batch, return_tensors="pt", padding=True).to(device))
        .logits[:, -1]
        .cpu()
    )
hf_probs = th.cat(hf_probs)
hf_probs = th.softmax(hf_probs, dim=-1)

print((tl_probs - hf_probs).abs().max(dim=-1).values.mean())

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model google/gemma-2b into HookedTransformer


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Français: "mot" - 中文: "詞"
Français: "position" - 中文: "地理位置"
Français: "porte" - 中文: "戶"
Français: "plate-forme" - 中文: "月台"
Français: "onde" - 中文: "海浪"
Français: "temps" - 中文: "

Français: "mot" - 中文: "詞"
Français: "position" - 中文: "地理位置"
Français: "porte" - 中文: "戶"
Français: "plate-forme" - 中文: "月台"
Français: "onde" - 中文: "

Français: "mot" - 中文: "詞"
Français: "position" - 中文: "地理位置"
Français: "porte" - 中文: "戶"
Français: "plate-forme" - 中文: "

Français: "mot" - 中文: "詞"
Français: "position" - 中文: "地理位置"
Français: "porte" - 中文: "

Français: "mot" - 中文: "詞"
Français: "position" - 中文: "

Français: "mot" - 中文: "



  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

tensor(0.3879)
